In [1]:
import utils_data as ut
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertForMaskedLM, BertTokenizer
tokenizerBERT = BertTokenizer.from_pretrained('pranav-s/MaterialsBERT')
modelBERT = BertForMaskedLM.from_pretrained('pranav-s/MaterialsBERT')

In [2]:
classes = {'POLYMER': 1,
           'ORGANIC': 2,
           'MONOMER': 3,
           'PROP_NAME': 4,
           'INORGANIC': 5,
           'MATERIAL_AMOUNT': 6,
           'POLYMER_FAMILY': 7,
           'PROP_VALUE': 8,
           'O': 0}
class NERBERTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = modelBERT.base_model
        self.linear = nn.Linear(768, len(classes))
        
    def forward(self, token):
        encoder_output= self.bert(token)
        linear_output = self.linear(encoder_output.last_hidden_state[0,:,:])
        class_output = F.softmax(linear_output, dim=1)
        return class_output

In [3]:
model = NERBERTModel()

In [4]:
data_list = ut.read_data('train.json')

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
epochs = 10
train_losses = []
for epoch in range(epochs):
    for data in data_list[:100]:
        token = ut.list2token(tokenizerBERT, data['words'])
        prediction = model(token)
        target = torch.tensor(ut.cat2digit(classes, data['ner']))
        loss = criterion(prediction, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         print(loss)
    train_losses.append(loss)
    print(f'epoch: {epoch:2}  loss: {loss.item():10.8f}')

epoch:  0  loss: 1.42592752
epoch:  1  loss: 1.42592382
epoch:  2  loss: 1.42592311
epoch:  3  loss: 1.42592180
epoch:  4  loss: 1.42592156
epoch:  5  loss: 1.42592061
epoch:  6  loss: 1.42592061
epoch:  7  loss: 1.42592025
epoch:  8  loss: 1.42592025
epoch:  9  loss: 1.42592013
